In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from yolov2 import YOLOv2D19 as YOLOv2
from detection_datasets import VOCDatasetV2
from loss import YOLOv2Loss
import pickle
with open('anchors_VOC0712trainval.pickle', 'rb') as handle:
    anchors = pickle.load(handle)

In [2]:
transforms = A.Compose([
    A.Resize(width=416, height=416),
    A.VerticalFlip(p=1.0),
    ToTensorV2()
], bbox_params=A.BboxParams(format='pascal_voc'))
train_set = VOCDatasetV2(devkit_path = '../../datasets/VOCdevkit/', scales=[13], anchors=anchors, transforms=transforms, 
                         dtype=torch.float32, device=torch.device('cpu'))

True ../../datasets/VOCdevkit/VOC2007\ImageSets\Main\trainval.txt
True ../../datasets/VOCdevkit/VOC2012\ImageSets\Main\trainval.txt


In [ ]:
train_loader = DataLoader(train_set, batch_size=64, shuffle=False)